In [3]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [4]:
def distance(instance1,instance2,k):
    distance=0;
    mm1=instance1[0]
    cm1=instance1[1]
    mm2=instance2[0] 
    cm2=instance2[1]
    distance=np.trace(np.dot(np.linalg.inv(cm2),cm1))
    distance+=(np.dot(np.dot((mm2-mm1).transpose(),np.linalg.inv(cm2)),mm2-mm1))
    distance+=np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-=k
    return distance

In [5]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x],instance,k) + distance(instance,trainingSet[x],k)
        distances.append((trainingSet[x][2], dist))
    
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [6]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]


In [7]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)


In [16]:
directory = "C:/Users/PC/Downloads/genres/"
f= open("C:/Users/PC/Documents/SGP-III/my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break 	
    for file in os.listdir(directory+folder):	
        try:
            (rate,sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix , covariance , i)
            pickle.dump(feature , f)
        except Exception as e:
            print('Got an exception: ',e,' in folder: ',folder,' filename: ',file)

f.close()


In [17]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open(filename , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break	

    for x in range(len(dataset)):
        if random.random() <split :			
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])	

trainingSet = []
testSet = []
loadDataset("C:/Users/PC/Documents/SGP-III/my.dat" , 0.66, trainingSet, testSet)

In [10]:
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 5))) 

accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)

0.6600660066006601


In [11]:
from collections import defaultdict
results=defaultdict(int)

i=1
for folder in os.listdir(directory):
    results[i]=folder
    i+=1
    
print(results)

defaultdict(<class 'int'>, {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 6: 'metal', 7: 'pop', 8: 'reggae', 9: 'rock'})


In [ ]:
f= open("C:/Users/PC/Documents/SGP-III/my_model.dat" ,'wb')

In [20]:
test_dir="C:/Users/PC/Music/"
test_file=test_dir + "song1.wav"

In [21]:
(rate,sig) = wav.read(test_file)
mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix , covariance)

In [22]:
pred=nearestClass(getNeighbors(dataset,feature,5))
print(results[pred])

hiphop
